<center>

## Segmenting and Clustering Neighbourhoods in Toronto
___

##### Install and Import Libraries

In [3]:
import pandas as pd

import matplotlib as mtp

import numpy as np

import requests 

from pandas.io.json import json_normalize

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


##### Import Data from Wikipedia :

In [4]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

df = pd.DataFrame(data[0])
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
df.shape

(180, 3)

##### Cleaning the table from 'Not assigned' values

In [6]:
df= df[df.Borough != 'Not assigned']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
df.shape

(103, 3)

##### Group neighbourdhoods and boroughs with identical postal codes

In [8]:
df = df.groupby(['Postal Code', 'Borough', 'Neighbourhood']).agg({'Postal Code':lambda x: ', '.join(tuple(x.tolist())),

                                     'Neighbourhood':lambda x: ', '.join(tuple(x.tolist()))}
                                   )
df.head(5)

,,,Postal Code,Neighbourhood
Postal Code,Borough,Neighbourhood,,
M1B,Scarborough,"Malvern, Rouge",M1B,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn,M1G,Woburn
M1H,Scarborough,Cedarbrae,M1H,Cedarbrae


##### Merging neighbourdhoods and boroughs with identical postal codes

In [9]:
df = df.groupby('Borough').agg({'Postal Code':'first', 
                             'Neighbourhood': ', '.join 
                              }).reset_index()
df.head(20)

,Borough,Postal Code,Neighbourhood
0,Central Toronto,M4N,"Lawrence Park, Davisville North, North Toronto..."
1,Downtown Toronto,M4W,"Rosedale, St. James Town, Cabbagetown, Church ..."
2,East Toronto,M4E,"The Beaches, The Danforth West, Riverdale, Ind..."
3,East York,M4B,"Parkview Hill, Woodbine Gardens, Woodbine Heig..."
4,Etobicoke,M8V,"New Toronto, Mimico South, Humber Bay Shores, ..."
5,Mississauga,M7R,Canada Post Gateway Processing Centre
6,North York,M2H,"Hillcrest Village, Fairview, Henry Farm, Oriol..."
7,Scarborough,M1B,"Malvern, Rouge, Rouge Hill, Port Union, Highla..."
8,West Toronto,M6H,"Dufferin, Dovercourt Village, Little Portugal,..."
9,York,M6C,"Humewood-Cedarvale, Caledonia-Fairbanks, Del R..."


##### Change order label list

In [10]:
columnsList=["Postal Code","Borough", "Neighbourhood"]
df=df.reindex(columns=columnsList)
df.head(20)

,Postal Code,Borough,Neighbourhood
0,M4N,Central Toronto,"Lawrence Park, Davisville North, North Toronto..."
1,M4W,Downtown Toronto,"Rosedale, St. James Town, Cabbagetown, Church ..."
2,M4E,East Toronto,"The Beaches, The Danforth West, Riverdale, Ind..."
3,M4B,East York,"Parkview Hill, Woodbine Gardens, Woodbine Heig..."
4,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores, ..."
5,M7R,Mississauga,Canada Post Gateway Processing Centre
6,M2H,North York,"Hillcrest Village, Fairview, Henry Farm, Oriol..."
7,M1B,Scarborough,"Malvern, Rouge, Rouge Hill, Port Union, Highla..."
8,M6H,West Toronto,"Dufferin, Dovercourt Village, Little Portugal,..."
9,M6C,York,"Humewood-Cedarvale, Caledonia-Fairbanks, Del R..."


In [11]:
df_geo_coordinate = pd.read_csv('Geospatial_Coordinates.csv')
df_geo_coordinate.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df_geo_coordinate.shape

(103, 3)

In [13]:
df_geo_coordinate.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_geo_coordinate.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M4N,Central Toronto,"Lawrence Park, Davisville North, North Toronto..."
1,M4W,Downtown Toronto,"Rosedale, St. James Town, Cabbagetown, Church ..."
2,M4E,East Toronto,"The Beaches, The Danforth West, Riverdale, Ind..."
3,M4B,East York,"Parkview Hill, Woodbine Gardens, Woodbine Heig..."
4,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores, ..."


In [21]:
df_merged = df.join(df_geo_coordinate.set_index('PostalCode'), on='PostalCode')
df_merged

KeyError: "None of ['Postal_Code'] are in the columns"